In [1]:
import torch
import torch.multiprocessing as mp
import torch.nn as nn
from qpth.qp import QPFunction

import numpy as np
import random

import data_generator
import params_newsvendor as params

from sklearn.preprocessing import StandardScaler

from model import VariationalLayer, VariationalNet, StandardNet, VariationalNet2

from train import TrainDecoupled, TrainCombined

from train_normflow import TrainFlowDecoupled, TrainFlowCombined

import joblib


import pdb

In [2]:
is_cuda = False
dev = torch.device('cpu')  
if torch.cuda.is_available():
    is_cuda = True
    dev = torch.device('cuda')

In [3]:
# Setting the seeds to allow replication
# Changing the seed might require hyperparameter tuning again
# Because it changes the deterministic parameters
seed_number = 0
np.random.seed(seed_number)
torch.manual_seed(seed_number)
random.seed(seed_number)

In [4]:
# Setting parameters (change if necessary)
N = 8000 # Total data size
N_train = 5000 # Training data size
N_SAMPLES = 2 # Sampling size while training
BATCH_SIZE_LOADER = 16 # Standard batch size
EPOCHS = 150 

In [5]:
nl = 1

In [6]:
# Data manipulation
N_valid = N - N_train
X, Y_original = data_generator.data_4to8(N_train, noise_level=nl)

In [7]:
# Output normalization
scaler = StandardScaler()
scaler.fit(Y_original)
tmean = torch.tensor(scaler.mean_)
tstd = torch.tensor(scaler.scale_)
joblib.dump(scaler, 'scaler_multi.gz')

['scaler_multi.gz']

In [8]:
def inverse_transform(yy):
    return yy*tstd + tmean

Y = scaler.transform(Y_original).copy()
X = torch.tensor(X, dtype=torch.float32)
Y = torch.tensor(Y, dtype=torch.float32)

In [9]:
data_train_original = data_generator.ArtificialDataset(X, Y_original)
training_loader_original = torch.utils.data.DataLoader(
    data_train_original, batch_size=BATCH_SIZE_LOADER,
    shuffle=False, num_workers=mp.cpu_count())

data_train = data_generator.ArtificialDataset(X, Y)
training_loader = torch.utils.data.DataLoader(
    data_train, batch_size=BATCH_SIZE_LOADER,
    shuffle=False, num_workers=mp.cpu_count())

X_val, Y_val_original = data_generator.data_4to8(N_valid, noise_level=nl)
Y_val = scaler.transform(Y_val_original).copy()
X_val = torch.tensor(X_val, dtype=torch.float32)
Y_val_original = torch.tensor(Y_val_original, dtype=torch.float32)
Y_val = torch.tensor(Y_val, dtype=torch.float32)

data_valid_original = data_generator.ArtificialDataset(X_val, Y_val_original)
validation_loader_original = torch.utils.data.DataLoader(
    data_valid_original, batch_size=BATCH_SIZE_LOADER,
    shuffle=False, num_workers=mp.cpu_count())

data_valid = data_generator.ArtificialDataset(X_val, Y_val)
validation_loader = torch.utils.data.DataLoader(
    data_valid, batch_size=BATCH_SIZE_LOADER,
    shuffle=False, num_workers=mp.cpu_count())

input_size = X.shape[1]
output_size = Y.shape[1]

In [10]:
class SolveNewsvendorWithKKT():
    def __init__(self, params_t, n_samples):
        super(SolveNewsvendorWithKKT, self).__init__()
        
        self.params_t = params_t
        
        n_items = len(params_t['c'])
        self.n_items = n_items  
        self.n_samples = n_samples
            
        # Torch parameters for KKT         
        ident = torch.eye(n_items)
        ident_samples = torch.eye(n_items*n_samples)
        ident3 = torch.eye(n_items + 2*n_items*n_samples)
        zeros_matrix = torch.zeros((n_items*n_samples, n_items*n_samples))
        zeros_array = torch.zeros(n_items*n_samples)
        ones_array = torch.ones(n_items*n_samples)
             
        self.Q = torch.diag(
            torch.hstack(
                (
                    params_t['q'], 
                    (1/n_samples)*params_t['qs'].repeat_interleave(n_samples), 
                    (1/n_samples)*params_t['qw'].repeat_interleave(n_samples)
                )
            )).to(dev)
        
        
        self.lin = torch.hstack(
                                (
                                    params_t['c'], 
                                    (1/n_samples)*params_t['cs'].repeat_interleave(n_samples), 
                                    (1/n_samples)*params_t['cw'].repeat_interleave(n_samples)
                                )).to(dev)
             
            
        shortage_ineq = torch.hstack(
            (
                -ident.repeat_interleave(n_samples, 0), 
                -ident_samples, 
                zeros_matrix
            )
        )  
        
        
        excess_ineq = torch.hstack(
            (
                ident.repeat_interleave(n_samples, 0), 
                zeros_matrix, 
                -ident_samples
            )
        )
        
        
        price_ineq = torch.hstack(
            (
                params_t['pr'], 
                zeros_array, 
                zeros_array
            )
        )
        
        
        positive_ineq = -ident3
        
        
        self.ineqs = torch.vstack(
            (
                shortage_ineq, 
                excess_ineq, 
                price_ineq, 
                positive_ineq
            )
        ).to(dev)
 
        self.uncert_bound = torch.hstack((-ones_array, ones_array)).to(dev)
        
        self.determ_bound = torch.tensor([params_t['B']]) 
        
        self.determ_bound = torch.hstack((self.determ_bound, 
                                          torch.zeros(n_items), 
                                          torch.zeros(n_items*n_samples), 
                                          torch.zeros(n_items*n_samples))).to(dev)
        
        
        
    def forward(self, y):
        """
        Applies the qpth solver for all batches and allows backpropagation.
        Formulation based on Priya L. Donti, Brandon Amos, J. Zico Kolter (2017).
        Note: The quadratic terms (Q) are used as auxiliar terms only to allow the backpropagation through the 
        qpth library from Amos and Kolter. 
        We will set them as a small percentage of the linear terms (Wilder, Ewing, Dilkina, Tambe, 2019)
        """
        
        batch_size, n_samples_items = y.size()
                
        assert self.n_samples*self.n_items == n_samples_items 

        Q = self.Q
        Q = Q.expand(batch_size, Q.size(0), Q.size(1))
        
        lin = self.lin
        lin = lin.expand(batch_size, lin.size(0))

        ineqs = torch.unsqueeze(self.ineqs, dim=0)
        ineqs = ineqs.expand(batch_size, ineqs.shape[1], ineqs.shape[2])       

        uncert_bound = (self.uncert_bound*torch.hstack((y, y)))
        determ_bound = self.determ_bound.unsqueeze(dim=0).expand(
            batch_size, self.determ_bound.shape[0])
        bound = torch.hstack((uncert_bound, determ_bound))     
        
        e = torch.DoubleTensor().to(dev)
        
        argmin = QPFunction(verbose=-1)\
            (Q.double(), lin.double(), ineqs.double(), 
             bound.double(), e, e).double()
            
        return argmin[:,:n_items]
    
    def cost_per_item(self, Z, Y):
        return params_t['q'].to(dev)*Z.to(dev)**2 \
            + self.params_t['qs'].to(dev)*(torch.max(torch.zeros((self.n_items)).to(dev),Y.to(dev)-Z.to(dev)))**2 \
            + self.params_t['qw'].to(dev)*(torch.max(torch.zeros((self.n_items)).to(dev),Z.to(dev)-Y.to(dev)))**2 \
            + self.params_t['c'].to(dev)*Z.to(dev) \
            + self.params_t['cs'].to(dev)*torch.max(torch.zeros((self.n_items)).to(dev),Y.to(dev)-Z.to(dev)) \
            + self.params_t['cw'].to(dev)*torch.max(torch.zeros((self.n_items)).to(dev),Z.to(dev)-Y.to(dev))

    
    def reshape_outcomes(self, y_pred):
                
        if len(y_pred.shape) == 2:
            y_pred = y_pred.unsqueeze(0)

        n_samples = y_pred.shape[0]
        batch_size = y_pred.shape[1]
        #n_items = y_pred.shape[2]
        y_pred = y_pred.permute((1, 2, 0)).reshape((batch_size, n_samples*self.n_items))
        return y_pred
    
    def calc_f_por_item(self, y_pred, y):
        #pdb.set_trace()
        y_pred = self.reshape_outcomes(y_pred)
        z_star =  self.forward(y_pred)
        f_per_item = self.cost_per_item(z_star, y)
        return f_per_item

    def calc_f_per_day(self, y_pred, y):
        f_per_item = self.calc_f_por_item(y_pred, y)
        f = torch.sum(f_per_item, 1)
        return f

    def end_loss(self, y_pred, y):
        f = self.calc_f_per_day(y_pred, y)
        f_total = torch.mean(f)
        return f_total
    
    def end_loss_dist(self, y_pred, y):
        f = self.calc_f_per_day(y_pred, y)
        f_total = torch.mean(f)
        return f_total

cost_per_item = lambda Z, Y : params_t['q'].to(dev)*Z.to(dev)**2 \
                            + params_t['qs'].to(dev)*(torch.max(torch.zeros((n_items)).to(dev),Y.to(dev)-Z.to(dev)))**2 \
                            + params_t['qw'].to(dev)*(torch.max(torch.zeros((n_items)).to(dev),Z.to(dev)-Y.to(dev)))**2 \
                            + params_t['c'].to(dev)*Z.to(dev) \
                            + params_t['cs'].to(dev)*torch.max(torch.zeros((n_items)).to(dev),Y.to(dev)-Z.to(dev)) \
                            + params_t['cw'].to(dev)*torch.max(torch.zeros((n_items)).to(dev),Z.to(dev)-Y.to(dev))


def reshape_outcomes(y_pred):
    n_samples = y_pred.shape[0]
    batch_size = y_pred.shape[1]
    n_items = y_pred.shape[2]

    y_pred = y_pred.permute((1, 2, 0)).reshape((batch_size, n_samples*n_items))

    return y_pred

def calc_f_por_item(y_pred, y):
    y_pred = reshape_outcomes(y_pred)
    z_star =  argmin_solver(y_pred)
    f_per_item = cost_per_item(z_star, y)
    return f_per_item

def calc_f_per_day(y_pred, y):
    f_per_item = calc_f_por_item(y_pred, y)
    f = torch.sum(f_per_item, 1)
    return f

def cost_fn(y_pred, y):
    f = calc_f_per_day(y_pred, y)
    f_total = torch.mean(f)
    return f_total

In [16]:
h_ann_dec = StandardNet(input_size, output_size, 0).to(dev)
h_bnn_dec = VariationalNet2(N_SAMPLES, input_size, output_size, 1.0).to(dev)

h_ann_com = StandardNet(input_size, output_size, 0).to(dev)
h_bnn_com = VariationalNet2(N_SAMPLES, input_size, output_size, 1.0).to(dev)

opt_h_ann_dec = torch.optim.Adam(h_ann_dec.parameters(), lr=0.001)
opt_h_bnn_dec = torch.optim.Adam(h_bnn_dec.parameters(), lr=0.001)

opt_h_ann_com = torch.optim.Adam(h_ann_com.parameters(), lr=0.005)
opt_h_bnn_com = torch.optim.Adam(h_bnn_com.parameters(), lr=0.005)

mse_loss = nn.MSELoss(reduction='none')

In [12]:
n_items = output_size
params_t, _ = params.get_params(n_items, seed_number)

# Construct the solver
cn_constrained = SolveNewsvendorWithKKT(params_t, 1)
cn_constrained_dist = SolveNewsvendorWithKKT(params_t, N_SAMPLES*4)

In [18]:
train_ANN_dec = TrainDecoupled(
                    bnn = False,
                    model=h_ann_dec,
                    opt=opt_h_ann_dec,
                    loss_data=mse_loss,
                    K=0.0,
                    training_loader=training_loader,
                    validation_loader=validation_loader
                )

train_ANN_dec.train(EPOCHS=EPOCHS)
model_ann_dec = train_ANN_dec.model

------------------EPOCH 1------------------
DATA LOSS 	 train 0.449 valid 0.329
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.45 valid 0.33
------------------EPOCH 2------------------
DATA LOSS 	 train 0.27 valid 0.264
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.27 valid 0.26
------------------EPOCH 3------------------
DATA LOSS 	 train 0.228 valid 0.242
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.23 valid 0.24
------------------EPOCH 4------------------
DATA LOSS 	 train 0.209 valid 0.229
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.21 valid 0.23
------------------EPOCH 5------------------
DATA LOSS 	 train 0.194 valid 0.217
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.19 valid 0.22
------------------EPOCH 6------------------
DATA LOSS 	 train 0.18 valid 0.2
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.18 valid 0.2
------------------EPOCH 7------------------
DATA LOSS 	 train 0.162 valid 0.181
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.16 valid 0.

DATA LOSS 	 train 0.054 valid 0.087
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.05 valid 0.09
------------------EPOCH 59------------------
DATA LOSS 	 train 0.054 valid 0.088
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.05 valid 0.09
------------------EPOCH 60------------------
DATA LOSS 	 train 0.053 valid 0.088
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.05 valid 0.09
------------------EPOCH 61------------------
DATA LOSS 	 train 0.053 valid 0.089
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.05 valid 0.09
------------------EPOCH 62------------------
DATA LOSS 	 train 0.053 valid 0.088
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.05 valid 0.09
------------------EPOCH 63------------------
DATA LOSS 	 train 0.053 valid 0.088
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.05 valid 0.09
------------------EPOCH 64------------------
DATA LOSS 	 train 0.052 valid 0.089
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.05 valid 0.09
------------------EPOCH 65----

DATA LOSS 	 train 0.044 valid 0.088
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.04 valid 0.09
------------------EPOCH 116------------------
DATA LOSS 	 train 0.043 valid 0.088
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.04 valid 0.09
------------------EPOCH 117------------------
DATA LOSS 	 train 0.042 valid 0.088
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.04 valid 0.09
------------------EPOCH 118------------------
DATA LOSS 	 train 0.042 valid 0.088
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.04 valid 0.09
------------------EPOCH 119------------------
DATA LOSS 	 train 0.042 valid 0.09
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.04 valid 0.09
------------------EPOCH 120------------------
DATA LOSS 	 train 0.042 valid 0.088
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.04 valid 0.09
------------------EPOCH 121------------------
DATA LOSS 	 train 0.041 valid 0.09
KL LOSS 	 train 0.0 valid 0.0
ELBO LOSS 	 train 0.04 valid 0.09
------------------EPOCH 12

In [19]:
train_ANN_com = TrainCombined(
                    bnn = False,
                    model=h_ann_com,
                    opt=opt_h_ann_com,
                    training_loader=training_loader_original,
                    validation_loader=validation_loader_original,
                    OP = cn_constrained
                )

train_ANN_com.train(EPOCHS=35)
model_ann_com = train_ANN_com.model

------------------EPOCH 1------------------
END LOSS 	 train 35054.674 valid 33125.471
------------------EPOCH 2------------------
END LOSS 	 train 32804.396 valid 32330.899
------------------EPOCH 3------------------
END LOSS 	 train 32314.45 valid 31022.274
------------------EPOCH 4------------------
END LOSS 	 train 30408.811 valid 30117.512
------------------EPOCH 5------------------
END LOSS 	 train 29644.298 valid 29263.458
------------------EPOCH 6------------------
END LOSS 	 train 29068.86 valid 28970.194
------------------EPOCH 7------------------
END LOSS 	 train 28831.273 valid 28878.379
------------------EPOCH 8------------------
END LOSS 	 train 28700.526 valid 28692.009
------------------EPOCH 9------------------
END LOSS 	 train 28543.471 valid 28502.843
------------------EPOCH 10------------------
END LOSS 	 train 28419.366 valid 28642.384
------------------EPOCH 11------------------
END LOSS 	 train 28308.034 valid 28393.69
------------------EPOCH 12------------------

In [20]:
train_BNN_dec = TrainDecoupled(
                    bnn = True,
                    model=h_bnn_dec,
                    opt=opt_h_bnn_dec,
                    loss_data=mse_loss,
                    K=0.0,
                    training_loader=training_loader_original,
                    validation_loader=validation_loader_original
                )

train_BNN_dec.train(EPOCHS=EPOCHS)
model_bnn_dec = train_BNN_dec.model

------------------EPOCH 1------------------


RuntimeError: Found dtype Double but expected Float

In [17]:
train_BNN_com = TrainCombined(
                    bnn = True,
                    model=h_bnn_com,
                    opt=opt_h_bnn_com,
                    training_loader=training_loader_original,
                    validation_loader=validation_loader_original,
                    OP = cn_constrained_dist
                )

train_BNN_com.train(EPOCHS=8)
model_bnn_com = train_BNN_com.model

------------------EPOCH 1------------------
END LOSS 	 train 67603.191 valid 64863.397
------------------EPOCH 2------------------
END LOSS 	 train 63983.753 valid 62837.398
------------------EPOCH 3------------------
END LOSS 	 train 62835.602 valid 61955.275
------------------EPOCH 4------------------
END LOSS 	 train 61977.076 valid 61275.862
------------------EPOCH 5------------------
END LOSS 	 train 61190.33 valid 60731.408
------------------EPOCH 6------------------
END LOSS 	 train 60356.31 valid 60510.741
------------------EPOCH 7------------------
END LOSS 	 train 59374.942 valid 58785.084
------------------EPOCH 8------------------
END LOSS 	 train 58795.948 valid 58552.18


In [21]:
trfl = TrainFlowDecoupled(steps = 5000, input_size=4, output_size=8)
pyx = trfl.train(X, Y, X_val, Y_val)
model_flow_dec = pyx

step: 0, train loss: 4.62412, val loss: 4.55984
step: 50, train loss: 3.4359, val loss: 3.4497
step: 100, train loss: 2.82206, val loss: 2.88951
step: 150, train loss: 2.48787, val loss: 2.57233
step: 200, train loss: 2.25889, val loss: 2.38194
step: 250, train loss: 2.10464, val loss: 2.24441
step: 300, train loss: 2.001, val loss: 2.14645
step: 350, train loss: 1.91222, val loss: 2.07915
step: 400, train loss: 1.85699, val loss: 2.02924
step: 450, train loss: 1.81211, val loss: 1.98739
step: 500, train loss: 1.77799, val loss: 1.95539
step: 550, train loss: 1.73116, val loss: 1.9221
step: 600, train loss: 1.69877, val loss: 1.89715
step: 650, train loss: 1.67398, val loss: 1.87381
step: 700, train loss: 1.65646, val loss: 1.84964
step: 750, train loss: 1.63558, val loss: 1.83198
step: 800, train loss: 1.60943, val loss: 1.80499
step: 850, train loss: 1.5891, val loss: 1.79062
step: 900, train loss: 1.55663, val loss: 1.76639
step: 950, train loss: 1.53143, val loss: 1.75044
step: 100

In [ ]:
# Training regression with FLOW
trfl = TrainFlowCombined(
    steps = 250, 
    input_size=4,
    output_size=8,
    lr=8e-3, 
    OP = cn_constrained_dist,
    n_samples=N_SAMPLES*4)
pyx = trfl.train(X, Y, X_val, Y_val)
model_flow_com = pyx

In [ ]:
# Propagating predictions to Newsvendor Problem


Y_pred_ANN_dec = model_ann_dec(X_val).unsqueeze(0)
Y_pred_ANN_dec = inverse_transform(Y_pred_ANN_dec)

Y_pred_ANN_com = model_ann_com(X_val).unsqueeze(0)
Y_pred_ANN_com = inverse_transform(Y_pred_ANN_com)

M = 4
model_bnn_dec.update_n_samples(n_samples=M)
Y_pred_BNN_dec = train_BNN_dec.model.forward_dist(X_val)
Y_pred_BNN_dec = inverse_transform(Y_pred_BNN_dec)

model_bnn_com.update_n_samples(n_samples=M)
Y_pred_BNN_com = train_BNN_com.model.forward_dist(X_val)
Y_pred_BNN_com = inverse_transform(Y_pred_BNN_com)
M = Y_pred_BNN_com.shape[0]

N = X_val.shape[0]
Y_pred_flow_dec = torch.zeros((M, N, n_items))
for i in range(0, N):
    Y_pred_flow_dec[:,i,:] = model_flow_dec.condition(X_val[i]).sample(torch.Size([M,])).squeeze()
Y_pred_flow_dec = inverse_transform(Y_pred_flow_dec)

#Y_pred_flow_com = torch.zeros((M, N, n_items))
#for i in range(0, N):
#    Y_pred_flow_com[:,i,:] = model_com.condition(X_val[i]).sample(torch.Size([M,])).squeeze()
#Y_pred_flow_com = inverse_transform(Y_pred_flow_com)

In [ ]:
mse_loss = nn.MSELoss()
print(mse_loss(Y_pred_ANN_dec.mean(axis=0), Y_val_original))
print(mse_loss(Y_pred_ANN_com.mean(axis=0), Y_val_original))

print(mse_loss(Y_pred_BNN_dec.mean(axis=0), Y_val_original))
print(mse_loss(Y_pred_BNN_com.mean(axis=0), Y_val_original))

print(mse_loss(Y_pred_flow_dec.mean(axis=0), Y_val_original))
#print(mse_loss(Y_pred_flow_com.mean(axis=0), Y_val_original))

In [ ]:
# Construct the solver
newsvendor_solve_kkt = SolveNewsvendorWithKKT(params_t, 1)
newsvendor_solve_kkt_M = SolveNewsvendorWithKKT(params_t, M)

In [ ]:
def optimize_final_block(Y_pred, cost):

    n_batches = int(np.ceil(Y_pred.shape[1]/BATCH_SIZE_LOADER))

    f_total = 0
    f_total_best = 0

    for b in range(0, n_batches):
        i_low = b*BATCH_SIZE_LOADER
        i_up = (b+1)*BATCH_SIZE_LOADER
        if b == n_batches-1:
            i_up = n_batches*Y_pred.shape[1]
        f_total += cost(Y_pred[:,i_low:i_up,:], Y_val_original[i_low:i_up,:])/n_batches
        print(b, f_total)

In [ ]:
optimize_final_block(Y_pred_ANN_dec, newsvendor_solve_kkt.end_loss)

In [ ]:
optimize_final_block(Y_pred_ANN_com, newsvendor_solve_kkt.end_loss)

In [ ]:
optimize_final_block(Y_pred_BNN_dec, newsvendor_solve_kkt_M.end_loss_dist)

In [ ]:
optimize_final_block(Y_pred_BNN_com, newsvendor_solve_kkt_M.end_loss_dist)

In [ ]:
optimize_final_block(Y_pred_flow_dec, newsvendor_solve_kkt_M.end_loss_dist)

In [32]:
def argmin_solver(Y_pred):
            z_star = newsvendor_solve_kkt_M.forward(Y_pred)
            return z_star

optimize_final_block(Y_pred_BNN_dec, True)

0 tensor(317.2303, dtype=torch.float64, grad_fn=<AddBackward0>)
1 tensor(545.6716, dtype=torch.float64, grad_fn=<AddBackward0>)
2 tensor(868.6623, dtype=torch.float64, grad_fn=<AddBackward0>)
3 tensor(1180.7642, dtype=torch.float64, grad_fn=<AddBackward0>)
4 tensor(1423.6939, dtype=torch.float64, grad_fn=<AddBackward0>)
5 tensor(1697.4194, dtype=torch.float64, grad_fn=<AddBackward0>)
6 tensor(2042.8964, dtype=torch.float64, grad_fn=<AddBackward0>)
7 tensor(2337.1373, dtype=torch.float64, grad_fn=<AddBackward0>)
8 tensor(2586.9145, dtype=torch.float64, grad_fn=<AddBackward0>)
9 tensor(2910.9963, dtype=torch.float64, grad_fn=<AddBackward0>)
10 tensor(3173.0135, dtype=torch.float64, grad_fn=<AddBackward0>)
11 tensor(3517.5267, dtype=torch.float64, grad_fn=<AddBackward0>)
12 tensor(3846.5958, dtype=torch.float64, grad_fn=<AddBackward0>)
13 tensor(4144.6255, dtype=torch.float64, grad_fn=<AddBackward0>)
14 tensor(4474.1223, dtype=torch.float64, grad_fn=<AddBackward0>)
15 tensor(4729.8327, dt

In [34]:
optimize_final_block(Y_pred_BNN_com, True)

0 tensor(343.3991, dtype=torch.float64, grad_fn=<AddBackward0>)
1 tensor(613.6026, dtype=torch.float64, grad_fn=<AddBackward0>)
2 tensor(971.4962, dtype=torch.float64, grad_fn=<AddBackward0>)
3 tensor(1319.0793, dtype=torch.float64, grad_fn=<AddBackward0>)
4 tensor(1599.1197, dtype=torch.float64, grad_fn=<AddBackward0>)
5 tensor(1910.8205, dtype=torch.float64, grad_fn=<AddBackward0>)
6 tensor(2283.6338, dtype=torch.float64, grad_fn=<AddBackward0>)
7 tensor(2615.8299, dtype=torch.float64, grad_fn=<AddBackward0>)
8 tensor(2905.5934, dtype=torch.float64, grad_fn=<AddBackward0>)
9 tensor(3260.9891, dtype=torch.float64, grad_fn=<AddBackward0>)
10 tensor(3561.1766, dtype=torch.float64, grad_fn=<AddBackward0>)
11 tensor(3937.5485, dtype=torch.float64, grad_fn=<AddBackward0>)
12 tensor(4305.7485, dtype=torch.float64, grad_fn=<AddBackward0>)
13 tensor(4645.9651, dtype=torch.float64, grad_fn=<AddBackward0>)
14 tensor(5018.1453, dtype=torch.float64, grad_fn=<AddBackward0>)
15 tensor(5323.1843, dt

In [35]:
optimize_final_block(Y_pred_flow_dec, True)

0 tensor(306.6597, dtype=torch.float64)
1 tensor(526.6195, dtype=torch.float64)
2 tensor(839.7867, dtype=torch.float64)
3 tensor(1143.7093, dtype=torch.float64)
4 tensor(1373.5122, dtype=torch.float64)
5 tensor(1641.9703, dtype=torch.float64)
6 tensor(1981.2830, dtype=torch.float64)
7 tensor(2268.6638, dtype=torch.float64)
8 tensor(2510.6073, dtype=torch.float64)
9 tensor(2829.9765, dtype=torch.float64)
10 tensor(3081.1508, dtype=torch.float64)
11 tensor(3421.6138, dtype=torch.float64)
12 tensor(3745.0731, dtype=torch.float64)
13 tensor(4040.5331, dtype=torch.float64)
14 tensor(4363.6344, dtype=torch.float64)
15 tensor(4608.8839, dtype=torch.float64)
16 tensor(4874.0525, dtype=torch.float64)
17 tensor(5140.8082, dtype=torch.float64)
18 tensor(5521.7318, dtype=torch.float64)
19 tensor(5767.2329, dtype=torch.float64)
20 tensor(6050.3973, dtype=torch.float64)
21 tensor(6288.7275, dtype=torch.float64)
22 tensor(6606.3052, dtype=torch.float64)
23 tensor(6863.0680, dtype=torch.float64)
24 te

In [36]:
optimize_final_block(Y_pred_flow_com, True)

0 tensor(310.6956, dtype=torch.float64)
1 tensor(536.3654, dtype=torch.float64)
2 tensor(858.5078, dtype=torch.float64)
3 tensor(1164.1711, dtype=torch.float64)
4 tensor(1398.0356, dtype=torch.float64)
5 tensor(1666.5549, dtype=torch.float64)
6 tensor(2011.1619, dtype=torch.float64)
7 tensor(2303.1385, dtype=torch.float64)
8 tensor(2545.9053, dtype=torch.float64)
9 tensor(2865.7012, dtype=torch.float64)
10 tensor(3115.8789, dtype=torch.float64)
11 tensor(3459.9512, dtype=torch.float64)
12 tensor(3786.4262, dtype=torch.float64)
13 tensor(4080.6408, dtype=torch.float64)
14 tensor(4406.1084, dtype=torch.float64)
15 tensor(4654.1746, dtype=torch.float64)
16 tensor(4919.8712, dtype=torch.float64)
17 tensor(5187.0698, dtype=torch.float64)
18 tensor(5564.7599, dtype=torch.float64)
19 tensor(5817.6144, dtype=torch.float64)
20 tensor(6110.3205, dtype=torch.float64)
21 tensor(6349.8374, dtype=torch.float64)
22 tensor(6668.7443, dtype=torch.float64)
23 tensor(6925.2494, dtype=torch.float64)
24 te

In [30]:
def argmin_solver(y_pred):
    z_star = newsvendor_solve_kkt.forward(y_pred)
    return z_star

n_batches = int(np.ceil(Y_pred_ANN_dec.shape[1]/BATCH_SIZE_LOADER))

f_total = 0
f_total_best = 0

for b in range(0, n_batches):
    i_low = b*BATCH_SIZE_LOADER
    i_up = (b+1)*BATCH_SIZE_LOADER
    if b == n_batches-1:
        i_up = n_batches*Y_pred_ANN_dec.shape[1]
    f_total += cost_fn(Y_pred_ANN_dec[:,i_low:i_up,:], Y_val_original[i_low:i_up,:])/n_batches
    print(b, f_total)

NameError: name 'Y_pred_ANN' is not defined

In [24]:
def argmin_solver(y_pred):
    z_star = newsvendor_solve_kkt_M.forward(y_pred)
    return z_star

n_batches = int(np.ceil(Y_pred_BNN.shape[1]/BATCH_SIZE_LOADER))

f_total = 0
f_total_best = 0

for b in range(0, n_batches):
    i_low = b*BATCH_SIZE_LOADER
    i_up = (b+1)*BATCH_SIZE_LOADER
    if b == n_batches-1:
        i_up = n_batches*Y_pred_BNN.shape[1]
    f_total += cost_fn(Y_pred_BNN[:,i_low:i_up,:], Y_val_original[i_low:i_up,:])/n_batches
    print(b, f_total)

0 tensor(336.8555, dtype=torch.float64, grad_fn=<AddBackward0>)
1 tensor(594.3570, dtype=torch.float64, grad_fn=<AddBackward0>)
2 tensor(939.0217, dtype=torch.float64, grad_fn=<AddBackward0>)
3 tensor(1272.9802, dtype=torch.float64, grad_fn=<AddBackward0>)
4 tensor(1544.8554, dtype=torch.float64, grad_fn=<AddBackward0>)
5 tensor(1846.9580, dtype=torch.float64, grad_fn=<AddBackward0>)
6 tensor(2217.8957, dtype=torch.float64, grad_fn=<AddBackward0>)
7 tensor(2530.7089, dtype=torch.float64, grad_fn=<AddBackward0>)
8 tensor(2803.0723, dtype=torch.float64, grad_fn=<AddBackward0>)
9 tensor(3155.3992, dtype=torch.float64, grad_fn=<AddBackward0>)
10 tensor(3440.7913, dtype=torch.float64, grad_fn=<AddBackward0>)
11 tensor(3810.9939, dtype=torch.float64, grad_fn=<AddBackward0>)
12 tensor(4165.9960, dtype=torch.float64, grad_fn=<AddBackward0>)
13 tensor(4488.8725, dtype=torch.float64, grad_fn=<AddBackward0>)
14 tensor(4842.5601, dtype=torch.float64, grad_fn=<AddBackward0>)
15 tensor(5129.6196, dt

In [25]:
def argmin_solver(y_pred):
    z_star = newsvendor_solve_kkt_M.forward(y_pred)
    return z_star

n_batches = int(np.ceil(Y_pred_flow.shape[1]/BATCH_SIZE_LOADER))

f_total = 0
f_total_best = 0

for b in range(0, n_batches):
    i_low = b*BATCH_SIZE_LOADER
    i_up = (b+1)*BATCH_SIZE_LOADER
    if b == n_batches-1:
        i_up = n_batches*Y_pred_flow.shape[1]
    f_total += cost_fn(Y_pred_flow[:,i_low:i_up,:], Y_val_original[i_low:i_up,:])/n_batches
    print(b, f_total)

0 tensor(312.2514, dtype=torch.float64)
1 tensor(543.1643, dtype=torch.float64)
2 tensor(868.0362, dtype=torch.float64)
3 tensor(1178.5969, dtype=torch.float64)
4 tensor(1417.3065, dtype=torch.float64)
5 tensor(1686.3286, dtype=torch.float64)
6 tensor(2031.0811, dtype=torch.float64)
7 tensor(2325.3115, dtype=torch.float64)
8 tensor(2574.8363, dtype=torch.float64)
9 tensor(2899.7800, dtype=torch.float64)
10 tensor(3151.4408, dtype=torch.float64)
11 tensor(3497.7524, dtype=torch.float64)
12 tensor(3825.7398, dtype=torch.float64)
13 tensor(4123.1164, dtype=torch.float64)
14 tensor(4450.7636, dtype=torch.float64)
15 tensor(4703.3666, dtype=torch.float64)
16 tensor(4968.6834, dtype=torch.float64)
17 tensor(5239.2102, dtype=torch.float64)
18 tensor(5619.9611, dtype=torch.float64)
19 tensor(5873.2957, dtype=torch.float64)
20 tensor(6163.6674, dtype=torch.float64)
21 tensor(6407.2110, dtype=torch.float64)
22 tensor(6728.7920, dtype=torch.float64)
23 tensor(6991.6152, dtype=torch.float64)
24 te

In [24]:
Y_val_original.shape

torch.Size([3000, 8])

In [26]:
def argmin_solver(y_pred):
    z_star = newsvendor_solve_kkt.forward(y_pred)
    return z_star

n_batches = int(np.ceil(Y_val_original.shape[0]/BATCH_SIZE_LOADER))

f_total = 0
f_total_best = 0

for b in range(0, n_batches):
    i_low = b*BATCH_SIZE_LOADER
    i_up = (b+1)*BATCH_SIZE_LOADER
    if b == n_batches-1:
        i_up = n_batches*Y_val_original.shape[0]
    f_total += cost_fn(Y_val_original[i_low:i_up,:].unsqueeze(0), Y_val_original[i_low:i_up,:])/n_batches
    print(f_total)

tensor(277.8333, dtype=torch.float64)
tensor(466.7020, dtype=torch.float64)
tensor(756.3428, dtype=torch.float64)
tensor(1038.8198, dtype=torch.float64)
tensor(1241.8659, dtype=torch.float64)
tensor(1478.8375, dtype=torch.float64)
tensor(1786.0079, dtype=torch.float64)
tensor(2042.8452, dtype=torch.float64)
tensor(2252.8012, dtype=torch.float64)
tensor(2544.8384, dtype=torch.float64)
tensor(2767.0454, dtype=torch.float64)
tensor(3077.3832, dtype=torch.float64)
tensor(3373.4957, dtype=torch.float64)
tensor(3640.8854, dtype=torch.float64)
tensor(3937.2814, dtype=torch.float64)
tensor(4155.9473, dtype=torch.float64)
tensor(4383.6695, dtype=torch.float64)
tensor(4621.0869, dtype=torch.float64)
tensor(4970.8880, dtype=torch.float64)
tensor(5186.6554, dtype=torch.float64)
tensor(5439.6769, dtype=torch.float64)
tensor(5652.0196, dtype=torch.float64)
tensor(5935.8175, dtype=torch.float64)
tensor(6155.1779, dtype=torch.float64)
tensor(6385.4286, dtype=torch.float64)
tensor(6632.4113, dtype=torc

In [27]:
(argmin_solver(reshape_outcomes(Y_val_original[0:50,:].unsqueeze(0)))*params_t['pr']).sum(1)

tensor([3777.6631, 3777.6631, 3777.6631, 3777.6631, 3067.9280, 3777.6631,
        3777.6631, 3777.6631, 3777.6631, 3777.6631, 3777.6631, 3777.6631,
        3777.6631, 3777.6631, 3496.4890, 3777.6631, 3777.6631, 3777.6631,
        3306.2300, 3777.6631, 3777.6631, 3659.2740, 3777.6631, 3777.6631,
        3777.6631, 3777.6631, 3710.0201, 3777.6631, 3777.6631, 3223.4030,
        3777.6631, 2657.2169, 3777.6631, 3338.9631, 3717.9910, 3777.6631,
        3777.6631, 3777.6631, 3777.6631, 3777.6631, 3777.6631, 3777.6631,
        3777.6631, 3584.3250, 3777.6631, 2956.0880, 3777.6631, 3777.6631,
        3777.6631, 3777.6631], dtype=torch.float64)

1.83 1.64 2.43 28104 27870 27787 13598
1.87 1.89 2.46 42607 42070 41574 28124
1.80 1.85 2.25 34329 33855 33395 15849


